In [3]:
from neo4j import GraphDatabase
import logging
from neo4j.exceptions import ServiceUnavailable
import pandas as pd

# Dataset

## ESCO Skill Dataset

In [4]:
skills = pd.read_csv('../data/all_skills.csv')[['concept_uri','preferred_label']]

## DIN PAS 1045 Course Description Datase

In [5]:
courses = pd.read_csv('../data/all_courses.csv')[['course_id','course_name','course_description']]

In [23]:
class App:

    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        # Don't forget to close the driver connection when you are finished with it
        self.driver.close()
        
    def create_skill(self, concept_uri, preferred_label, description):
        with self.driver.session() as session:
            # Write transactions allow the driver to handle retries and transient errors
            result = session.write_transaction(self._create_and_return_skill, 
                                               concept_uri, preferred_label, description)
            
    @staticmethod
    def _create_and_return_skill(tx, concept_uri, preferred_label, description):
        query = (
            "CREATE (skill:Skill {concept_uri: $concept_uri, \
            preferred_label: $preferred_label, description: $description})"
            "RETURN skill"
        )

        results = tx.run(query, concept_uri=skill["conceptUri"], 
                             preferred_label=skill["preferredLabel"], 
                             description=skill["description"])

        try:
            return [{"conceptUri": result["skill"]["concept_uri"], 
                     "preferredLabel": result["skill"]["preferred_label"],
                     "description": result["skill"]["description"]} 
                    for result in results]
            
        # Capture any errors along with the query and data for traceability
        except ServiceUnavailable as exception:
            logging.error("{query} raised an error: \n {exception}".format(
                query=query, exception=exception))
            raise
    
    def create_skills(self, csv_path):
        with self.driver.session() as session:
            result = session.write_transaction(self._create_skills, csv_path)
            
    @staticmethod
    def _create_skills(tx, csv_path):
        assert "http" in csv_path, "Loading csv files from local is \
        not possible due to security reasons, please use online files." 
        query = (
            # USING PERIODIC COMMIT 1000 
            "LOAD CSV WITH HEADERS FROM $csv_path AS skill \
            CREATE (:Skill {concept_uri: skill.concept_uri, \
            preferred_label: skill.preferred_label, description: skill.description})"
        )
        results = tx.run(query, csv_path=csv_path)
        
    def create_course(self, course_name, course_id, course_description):
        with self.driver.session() as session:
            # Write transactions allow the driver to handle retries and transient errors
            result = session.write_transaction(self._create_and_return_course, 
                                               course_name, course_id, course_description)
            
    @staticmethod
    def _create_and_return_course(tx, course_name, course_id, course_description):
        
        query = (
            "CREATE (course:Course {course_id: $course_id, course_name: \
            $course_name, course_description: $course_description})"
            "RETURN course"
        )

        results = tx.run(query, course_id=course["course_id"], 
                             course_name=course["course_name"], 
                             course_description=course["course_description"])

        try:
            return [{"course_id": result["course"]["course_id"], 
                     "course_name": result["course"]["course_name"],
                     "course_description": result["course"]["course_description"]} 
                    for result in results]
            
        # Capture any errors along with the query and data for traceability
        except ServiceUnavailable as exception:
            logging.error("{query} raised an error: \n {exception}".format(
                query=query, exception=exception))
            raise

In [24]:
if __name__ == "__main__":
    # Aura queries use an encrypted connection using the "neo4j+s" URI scheme
    uri = "neo4j+s://fbb398e7.databases.neo4j.io"
    user = "neo4j"
    password = "3bO4Zwc9eYOyZBQ_CHE49lqd7NKeLSySJAq4flxUtAA"
    app = App(uri, user, password)
    app.create_skills("https://github.com/ingastrelnikova/awt-pj-ss22-learn-tech-2/blob/main/data/all_skills.csv")
#     app.create_skill(conceptUri,preferredLabel,description)
#     app.create_course(course_id,course_name,course_description)
    app.close()

CREATE (ee:Person { name: "Emil", from: "Sweden", klout: 99 })

MATCH (ee:Person) WHERE ee.name = "Emil" RETURN ee;

CREATE (js:Person { name: "Johan", from: "Sweden", learn: "surfing" }),
(ir:Person { name: "Ian", from: "England", title: "author" }),
(rvb:Person { name: "Rik", from: "Belgium", pet: "Orval" }),
(ally:Person { name: "Allison", from: "California", hobby: "surfing" }),
(ee)-[:KNOWS {since: 2001}]->(js),(ee)-[:KNOWS {rating: 5}]->(ir),
(js)-[:KNOWS]->(ir),(js)-[:KNOWS]->(rvb),
(ir)-[:KNOWS]->(js),(ir)-[:KNOWS]->(ally),
(rvb)-[:KNOWS]->(ally)

MATCH
  (a:Person),
  (b:Movie)
WHERE a.name = 'Allison' AND b.title = 'The Matrix Revolutions'
CREATE (a)-[r:ACTED_IN]->(b)
RETURN type(r)

MATCH (n) DETACH DELETE n
MATCH (n) RETURN n

In [ ]:
movieId:ID,title,year:int,:LABEL
tt0133093,"The Matrix",1999,Movie
tt0234215,"The Matrix Reloaded",2003,Movie;Sequel
tt0242653,"The Matrix Revolutions",2003,Movie;Sequel